This will be the user space program to pretrain the ML model to
classify between malicious and normal packets. There are multiple ways
in which an ML model can be deployed and fit in the flow of the packets.

One way is using the XDP_AF sockets and sending data collected about the
packets from kernel space to the user space, where the ML model decides the
fate of the packet, does nothing (for drop) or sends packet to the corresponding
user space application after processing it in the required way. This could
also be implemented in other ways using perf buffer, polling etc.


Another approach would be to implement the ML model within the kernel and that
is the approach currently adopted.

Currently, we are implemented a basic in-kernel very simple (logisticRegression)NN
for classification of the packets. We are going to do quantization-aware-training in
the user space and we will store the quantized weights in a BPF map.

Since the kernel has the following restrictions :
    (1) limitations on the quantity of eBPF instructions and stack space,
    (2) prohibitions on unbounded loops, non-static global variables, variadic functions,
        multi-threaded programming, and floating-point representation, and
    (3) enforcement of array bound checks

We will go for a simple model, which would involve less complex calculations sa well

These weights can be updated as we collect more data and used to train the model
after certain intervals of time. Maybe at the end of every 24 hours. But this feature
will be implemented soon.


In [25]:
import glob
import os
import random
import zipfile

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.quantization import QuantStub, DeQuantStub

from tqdm import tqdm
from pathlib import Path

# Getting the CICIDS Dataset from Kaggle

In [2]:
!pip install kaggle

In [3]:
mkdir ~/.kaggle

In [4]:
cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d cicdataset/cicids2017

 91% 209M/230M [00:03<00:00, 89.7MB/s]
100% 230M/230M [00:03<00:00, 77.4MB/s]


In [7]:
!unzip cicids2017.zip

Archive:  cicids2017.zip
  inflating: MachineLearningCSV.md5  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  


# Loading the dataset into a dataframe

In [8]:
# list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

[]

In [9]:
# merging the files

# if local system use PATH = "dataset/archive/MachineLearningCSV/MachineLearningCVE/"
# else if on colab use PATH = "/content/MachineLearningCSV/MachineLearningCVE"
PATH = "/content/MachineLearningCSV/MachineLearningCVE"
joined_files = os.path.join(PATH, "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df_concat = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
print(df_concat)

          Destination Port   Flow Duration   Total Fwd Packets  \
0                      389       113095465                  48   
1                      389       113473706                  68   
2                        0       119945515                 150   
3                      443        60261928                   9   
4                       53             269                   2   
...                    ...             ...                 ...   
2830738                443          196135                  49   
2830739                443          378424                  49   
2830740                443          161800                  70   
2830741                443          142864                  50   
2830742                443          186928                  46   

          Total Backward Packets  Total Length of Fwd Packets  \
0                             24                         9668   
1                             40                        11364   
2           

In [10]:
df_concat.columns = df_concat.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_concat.head()

<ipython-input-10-4cb7c74aa1b2>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_concat.columns = df_concat.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
<ipython-input-10-4cb7c74aa1b2>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_concat.columns = df_concat.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,BENIGN
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,BENIGN
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,BENIGN
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [11]:
df_labels = df_concat['label']
df_labels.unique()

array(['BENIGN', 'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'Infiltration', 'FTP-Patator',
       'SSH-Patator', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed', 'DDoS', 'PortScan'],
      dtype=object)

In [12]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   destination_port             int64  
 1   flow_duration                int64  
 2   total_fwd_packets            int64  
 3   total_backward_packets       int64  
 4   total_length_of_fwd_packets  int64  
 5   total_length_of_bwd_packets  int64  
 6   fwd_packet_length_max        int64  
 7   fwd_packet_length_min        int64  
 8   fwd_packet_length_mean       float64
 9   fwd_packet_length_std        float64
 10  bwd_packet_length_max        int64  
 11  bwd_packet_length_min        int64  
 12  bwd_packet_length_mean       float64
 13  bwd_packet_length_std        float64
 14  flow_bytes/s                 float64
 15  flow_packets/s               float64
 16  flow_iat_mean                float64
 17  flow_iat_std                 float64
 18  flow_iat_max                 int64  
 19  

In [13]:
df_concat.corr()

<ipython-input-13-26fdf3a4e505>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_concat.corr()


,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
destination_port,1.000000,-0.151680,-0.004236,-0.003947,0.011145,-0.003082,0.097926,-0.045388,0.140220,0.128861,...,-0.003226,0.000897,-0.035562,-0.043717,-0.051859,-0.023194,-0.112585,0.010399,-0.108185,-0.114614
flow_duration,-0.151680,1.000000,0.020857,0.019670,0.065456,0.016186,0.273308,-0.105230,0.143689,0.234437,...,0.015942,-0.001357,0.189299,0.241060,0.294034,0.121171,0.768034,0.243154,0.779527,0.738328
total_fwd_packets,-0.004236,0.020857,1.000000,0.999070,0.365508,0.996993,0.009358,-0.002989,0.000032,0.001403,...,0.887387,-0.000184,0.039937,0.008329,0.030459,0.041283,0.001820,0.000809,0.001906,0.001670
total_backward_packets,-0.003947,0.019670,0.999070,1.000000,0.359451,0.994429,0.009039,-0.002600,-0.000333,0.001026,...,0.882566,0.000018,0.038963,0.006437,0.028602,0.041278,0.001425,0.000492,0.001456,0.001330
total_length_of_fwd_packets,0.011145,0.065456,0.365508,0.359451,1.000000,0.353762,0.197030,-0.000275,0.185262,0.159787,...,0.407448,-0.001209,0.101084,0.103326,0.126493,0.068325,0.022660,0.027064,0.026079,0.018634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
active_min,-0.023194,0.121171,0.041283,0.041278,0.068325,0.039069,0.105641,-0.025912,0.081170,0.094164,...,0.031394,-0.006834,0.905862,0.033874,0.584503,1.000000,0.118133,0.038302,0.122651,0.112880
idle_mean,-0.112585,0.768034,0.001820,0.001425,0.022660,0.000809,0.181135,-0.071304,0.127959,0.183139,...,0.000837,-0.000876,0.120171,0.036551,0.088904,0.118133,1.000000,0.150248,0.990387,0.990215
idle_std,0.010399,0.243154,0.000809,0.000492,0.027064,0.000105,0.178091,-0.029951,0.178462,0.191278,...,0.000721,-0.003720,0.070586,0.081435,0.070002,0.038302,0.150248,1.000000,0.283330,0.011609
idle_max,-0.108185,0.779527,0.001906,0.001456,0.026079,0.000797,0.199559,-0.073419,0.148402,0.203304,...,0.000929,-0.001407,0.132700,0.055300,0.102816,0.122651,0.990387,0.283330,1.000000,0.961812


In [15]:
def clean_df(df):
    # Remove the space before each feature names
    df.columns = df.columns.str.strip()
    print('dataset shape', df.shape)

    # This set of feature should have >= 0 values
    num = df._get_numeric_data()
    num[num < 0] = 0

    zero_variance_cols = []
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(zero_variance_cols, axis = 1, inplace = True)
    print('zero variance columns', zero_variance_cols, 'dropped')
    print('shape after removing zero variance columns:', df.shape)

    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    print(df.isna().any(axis = 1).sum(), 'rows dropped')
    df.dropna(inplace = True)
    print('shape after removing nan:', df.shape)

    # Drop duplicate rows
    df.drop_duplicates(inplace = True)
    print('shape after dropping duplicates:', df.shape)

    column_pairs = [(i, j) for i, j in combinations(df, 2) if df[i].equals(df[j])]
    ide_cols = []
    for column_pair in column_pairs:
        ide_cols.append(column_pair[1])
    df.drop(ide_cols, axis = 1, inplace = True)
    print('columns which have identical values', column_pairs, 'dropped')
    print('shape after removing identical value columns:', df.shape)
    return df
df_concat = clean_df(df_concat)

dataset shape (2830743, 79)
zero variance columns ['bwd_psh_flags', 'bwd_urg_flags', 'fwd_avg_bytes/bulk', 'fwd_avg_packets/bulk', 'fwd_avg_bulk_rate', 'bwd_avg_bytes/bulk', 'bwd_avg_packets/bulk', 'bwd_avg_bulk_rate'] dropped
shape after removing zero variance columns: (2830743, 71)
2867 rows dropped
shape after removing nan: (2827876, 71)
shape after dropping duplicates: (2520798, 71)
columns which have identical values [('total_fwd_packets', 'subflow_fwd_packets'), ('total_backward_packets', 'subflow_bwd_packets'), ('fwd_psh_flags', 'syn_flag_count'), ('fwd_urg_flags', 'cwe_flag_count'), ('fwd_header_length', 'fwd_header_length.1')] dropped
shape after removing identical value columns: (2520798, 66)


In [16]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520798 entries, 0 to 2830742
Data columns (total 66 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   destination_port             int64  
 1   flow_duration                int64  
 2   total_fwd_packets            int64  
 3   total_backward_packets       int64  
 4   total_length_of_fwd_packets  int64  
 5   total_length_of_bwd_packets  int64  
 6   fwd_packet_length_max        int64  
 7   fwd_packet_length_min        int64  
 8   fwd_packet_length_mean       float64
 9   fwd_packet_length_std        float64
 10  bwd_packet_length_max        int64  
 11  bwd_packet_length_min        int64  
 12  bwd_packet_length_mean       float64
 13  bwd_packet_length_std        float64
 14  flow_bytes/s                 float64
 15  flow_packets/s               float64
 16  flow_iat_mean                float64
 17  flow_iat_std                 float64
 18  flow_iat_max                 int64  
 19  

In [17]:
unique_vals = df_concat['label'].unique()
df_concat['label'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))),
           inplace=True)

In [18]:
mask = df_concat['label'] != 0
df_concat.loc[mask, 'label'] = 1

In [19]:
df_concat.describe()

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
count,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,...,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06,2.520798e+06
mean,8.690590e+03,1.659161e+07,1.028174e+01,1.157280e+01,6.119477e+02,1.814440e+04,2.312292e+02,1.920349e+01,6.350497e+01,7.732347e+01,...,2.588550e+01,9.157847e+04,4.619177e+04,1.720171e+05,6.546359e+04,9.337367e+06,5.657941e+05,9.763770e+06,8.892671e+06,1.688914e-01
std,1.901280e+04,3.523276e+07,7.944201e+02,1.056922e+03,1.058827e+04,2.398177e+06,7.563755e+02,6.079834e+01,1.955526e+02,2.968814e+02,...,6.525341e+00,6.866503e+05,4.165844e+05,1.085571e+06,6.111585e+05,2.484818e+07,4.874169e+06,2.561746e+07,2.458143e+07,3.746560e-01
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,2.080000e+02,2.000000e+00,1.000000e+00,1.200000e+01,6.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,5.062200e+04,2.000000e+00,2.000000e+00,6.600000e+01,1.560000e+02,4.000000e+01,2.000000e+00,3.625000e+01,0.000000e+00,...,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,5.333340e+06,6.000000e+00,5.000000e+00,3.320000e+02,9.970000e+02,2.020000e+02,3.700000e+01,5.200000e+01,7.419280e+01,...,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08,1.000000e+00


In [20]:
df_concat.head()

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,0
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,0
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,0
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,0
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,0


In [21]:
feature_list = ['destination_port', 'packet_length_mean','packet_length_std','packet_length_variance','average_packet_size','fwd_iat_mean','fwd_iat_std','fwd_iat_max']

In [22]:
from sklearn.model_selection import train_test_split

X = df_concat[feature_list]
y = df_concat['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        y = self.linear(x)
        y = torch.sigmoid(y)
        y = self.dequant(y)
        return y

In [24]:
def evaluate(model, data, criterion):
    loss = 0.0
    with torch.no_grad():
        for (x, y_target) in data:
            y = model(x)
            loss += criterion(y, y_target)
    return loss

In [26]:
X_train = torch.tensor(X_train.values, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.float)
X_test = torch.tensor(X_test.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)

In [27]:
del csv_files, df_concat, X, feature_list, joined_files, joined_list, mask, unique_vals, y, df_labels

In [28]:
_DIM_INPUT = 8  # 8
_DIM_OUTPUT = 1  # it's a binary classifier
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [29]:
model = LogisticRegression(_DIM_INPUT, _DIM_OUTPUT)

In [30]:
# Insert min-max observers in the model

model.qconfig = torch.ao.quantization.default_qconfig
model.train()
model_quantized = torch.ao.quantization.prepare_qat(model) # Insert observers
print(model_quantized)

LogisticRegression(
  (linear): Linear(
    in_features=8, out_features=1, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (dequant): DeQuantStub()
)


In [31]:
def train(x_data, y_data,model):

    criterion = torch.nn.BCELoss(reduction="sum")

    optimizer = torch.optim.Adagrad(model.parameters())

    for epoch in tqdm(range(1000)):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        y_pred = torch.reshape(model(x_data),(-1,))
        # Compute Loss
        loss = criterion(y_pred, y_data)
        # Backward pass
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print('epoch {}, loss {}'.format(epoch, loss.item() / len(x_data)))
    return
train(X_train, y_train,model_quantized)

  0%|          | 2/1000 [00:00<02:25,  6.84it/s]

epoch 0, loss 70.5285053638779


  1%|          | 12/1000 [00:01<03:01,  5.45it/s]

epoch 10, loss 37.20277808907697


  2%|▏         | 22/1000 [00:03<02:28,  6.58it/s]

...

 98%|█████████▊| 982/1000 [02:56<00:03,  5.44it/s]

epoch 980, loss 13.552981744864473


 99%|█████████▉| 992/1000 [02:58<00:01,  5.42it/s]

epoch 990, loss 13.282826169099263


100%|██████████| 1000/1000 [02:59<00:00,  5.56it/s]


In [32]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')

In [33]:
print_size_of_model(model_quantized)
print(f'Check statistics of the various layers')
print(model_quantized)

Size (KB): 4.194
Check statistics of the various layers
LogisticRegression(
  (linear): Linear(
    in_features=8, out_features=1, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-0.275702565908432, max_val=0.33877867460250854)
    (activation_post_process): MinMaxObserver(min_val=-33557856.0, max_val=17030178.0)
  )
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=0.0, max_val=120000000.0)
  )
  (dequant): DeQuantStub()
)


In [37]:
def acc(model_quantized):
    y_pred = model_quantized(X_test)
    num_correct = 0
    for i in range(len(y_pred)) :
        if y_pred[i] > 0.5 and y_test[i] == 1:
            num_correct+=1
        elif y_pred[i] <= 0.5 and y_test[i] == 0:
            num_correct+=1

    print("\nTest on %d samples: %d malicious pkts, predicted correctly %d or %.2f%%\n" % (\
        len(y_test), y_test.sum(), num_correct, num_correct * 100.0 / len(y_test)))

# testing before quantization

In [38]:
acc(model_quantized)


Test on 504160 samples: 85403 malicious pkts, predicted correctly 404926 or 80.32%



# Quantize the model using the statistics collected

In [39]:
model.eval()
model_quantized = torch.ao.quantization.convert(model_quantized)
print(f'Check statistics of the various layers')
print(model_quantized)

Check statistics of the various layers
LogisticRegression(
  (linear): QuantizedLinear(in_features=8, out_features=1, scale=398330.96875, zero_point=84, qscheme=torch.per_tensor_affine)
  (quant): Quantize(scale=tensor([944881.8750]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
)


# Print weights and size of the model after quantization

In [40]:
print('Weights after quantization')
print(torch.int_repr(model_quantized.linear.weight()))
print(model_quantized.linear.bias())
print("Size after quantization")
print_size_of_model(model_quantized)

Weights after quantization
tensor([[  0, -80, 106,  -9, -85, -52, 106, -45]], dtype=torch.int8)
Parameter containing:
tensor([0.0278], requires_grad=True)
Size after quantization
Size (KB): 2.854


# testing after quantization


In [41]:
acc(model_quantized)


Test on 504160 samples: 85403 malicious pkts, predicted correctly 418576 or 83.02%



## Saving the model weights


In [ ]:
torch.save(model.state_dict(), './src/model_weights.pth')

w = (torch.load('./src/model_weights.pth'))
print(w)

In [ ]:
for i in w:
  print("Key:",i)
  print("Value:",(w[i]))